In [1]:
!pip install selenium
!pip install webdriver-manager

from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
import datetime


In [2]:
#chromedriver_path = "C:/Users/elisa/Projet Python/chromedriver_win32/chromedriver.exe"
chromedriver_path = 'C:/Users/José/Documents/ENSAE Paris/2A/Python Pour le Data Scientist/chromedriver_win32/chromedriver.exe'

In [26]:
driver = webdriver.Chrome(chromedriver_path)


In [4]:
#depart = ["PAR", "PAR"]
#destination = ["SVO","ATH"]
#date_aller = ["2021-12-27"]
#date_retour = ["2022-01-03"]

In [5]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list

In [6]:
def remanier(compagnie):
    liste = []
    for i in range(len(compagnie)):
        if  ',' in compagnie[i] : 
            j= 0
            while compagnie[i][j]!=',' :
                j+=1
            liste.append(compagnie[i][:j])
            liste.append(compagnie[i][j+1:])
        else :
            liste.append(compagnie[i])
            liste.append(compagnie[i])
    return liste

In [7]:
def remanier_2(prix):
    liste = []
    for i in range(len(prix)):
        liste.append(prix[i])
        liste.append(prix[i])
    return liste

In [31]:
depart = ["PAR"]
Destinations = ["MAD", "SVO", "ATH", "OSL", "LON"]
date_aller = ["04/01/2022"]
date_retour = ["04/08/2022"]
date_a = pd.to_datetime(date_aller)
date_r = pd.to_datetime(date_retour)
result = pd.DataFrame([])

for k in range(26): 
    new_date_a = date_a + datetime.timedelta(days = 7*k)
    new_date_r = date_r + datetime.timedelta(days = 7*k)
    date_time_a = new_date_a.astype(str)
    date_time_r = new_date_r.astype(str)
    destination = ["ATH"]
    date_aller = [date_time_a]        
    date_retour = [date_time_r]
    url = f"https://www.kayak.fr/flights/{depart[0]}-{destination[0]}/{date_aller[0]}/{date_retour[0]}"
    driver.get(url)
    sleep(15)
        try:
        show_more_button = driver.find_element_by_xpath('//a[@class = "moreButton"]')
    except:
        input("Please solve the captcha then enter anything here to resume scraping.")
    while True:
        try:
            show_more_button.click()
            driver.implicitly_wait(10)
        except:
            break
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    compagnie = remanier(get_airlines(soup))
    total_escales = get_total_stops(soup)
    prix = remanier_2(get_price(soup))
    duree = get_duration(soup)
    df = pd.DataFrame({'Compagnie':compagnie, "Total d'escales":total_escales , 'Prix': prix, 'Duree': duree}, 
                        columns = ['Compagnie', "Total d'escales", 'Prix', 'Duree'])
    df["Destination"] = "Athenes"
    df["Distance"] = "2095.52"
    df["Date_aller"] = 20220401
    df["Date_retour"] = 20220408
    df["Date_aller"] = pd.to_datetime(df["Date_aller"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    df["Date_retour"] = pd.to_datetime(df["Date_retour"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    result = pd.concat([result, df], axis = 0)

ValueError: arrays must all be same length

In [ ]:
df['distance_entre_villes'] = df.append({"df['destination']=='LON'" : '342,76km' ,"df['destination']=='MAD'" : '1054km', "df['destination']=='OSL'" : '1344km', "df['destination']=='SVO'" : '2486,34km', "df['destination']=='ATH'" : '2095,52km' } , ignore_index=True)


In [35]:
result.to_excel('athenes.xlsx', index = False) 
result.sample(50)
#result["Total d'escales"] = result["Total d'escales"].replace("\n", "")
#result.sample(50)

,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
341,Plusieurs compagnies,\n1 escale,\n184 €,\n7h 35min,Athenes,2095.52,2022-04-29,2022-05-06
1263,Sky Express,\ndirect,\n201 €,\n3h 20min,Athenes,2095.52,2022-04-22,2022-04-29
1054,Aegean Airlines,\n1 escale,\n341 €,\n8h 40min,Athenes,2095.52,2022-04-29,2022-05-06
682,Lufthansa,\n1 escale,\n213 €,\n4h 35min,Athenes,2095.52,2022-05-20,2022-05-27
49,Aegean Airlines,\ndirect,\n194 €,\n3h 30min,Athenes,2095.52,2022-04-01,2022-04-08
879,Aegean Airlines,\ndirect,\n463 €,\n3h 30min,Athenes,2095.52,2022-05-13,2022-05-20
1450,Lufthansa,\n1 escale,\n185 €,\n4h 35min,Athenes,2095.52,2022-05-06,2022-05-13
686,Aegean Airlines,\ndirect,\n427 €,\n3h 05min,Athenes,2095.52,2022-04-22,2022-04-29
1380,Lufthansa,\n2 escales,\n445 €,\n13h 30min,Athenes,2095.52,2022-04-29,2022-05-06
1137,Aegean Airlines,\n1 escale,\n324 €,\n8h 50min,Athenes,2095.52,2022-05-20,2022-05-27


In [218]:
df = pd.read_excel('bdd.xlsx')



#df = df.replace('\xa','', regex=True)
df.sample(50)
df.dtypes
r = df.groupby("Total d'escales")
r.describe()


Unnamed: 0                                                 \
                     count          mean           std      min       25%   
Total d'escales                                                             
0 changement          12.0  51372.833333    255.697560  50998.0  51198.75   
1 changement           2.0  27622.500000      0.707107  27622.0  27622.25   
1 escale           31752.0  29007.640432  19391.492013     55.0  10877.75   
2 escales            786.0  36917.404580  19954.795953    527.0  16216.75   
3 escales              6.0  13885.000000    286.604257  13300.0  13998.50   
direct             28790.0  32335.365405  15350.331037      0.0  20850.25   

                                            Distance               \
                     50%       75%      max    count         mean   
Total d'escales                                                     
0 changement     51352.5  51628.25  51707.0     12.0   342.760000   
1 changement     27622.5  27622.75  27623.0      2.0  1054.000000   
1 escale         27366.5  46998.25  61347.0  31752.0  1593.209014   
2 escales        40446.0  56354.50  61184.0    786.0  1735.262850   
3 escales        14001.0  14003.50  14006.0      6.0  2486.340000   
direct           32253.5  45145.75  61346.0  28790.0  1315.629844   

                                                                            
                          std      min      25%      50%      75%      max  
Total d'escales                                                             
0 changement     5.937101e-14   342.76   342.76   342.76   342.76   342.76  
1 changement     0.000000e+00  1054.00  1054.00  1054.00  1054.00  1054.00  
1 escale         5.712684e+02  1054.00  1054.00  1344.00  2095.52  2486.34  
2 escales        5.599422e+02  1054.00  1054.00  2095.52  2095.52  2486.34  
3 escales        0.000000e+00  2486.34  2486.34  2486.34  2486.34  2486.34  
direct           5.351333e+02   342.76  1054.00  1054.00  1344.00  2486.34

In [256]:
#def escale(base):
df1 = df.copy()    
df1.set_index("Unnamed: 0", inplace = True)
df1.sample(50)
df1.columns

def formatage_escale(df1):
    a = pd.DataFrame([df1["Total d'escales"] != "direct ", df1["Total d'escales"] != "O changement "])
    df2 = df1.copy()
    df1["Total d'escales"] = a.all().astype(np.int64) + (df2["Total d'escales"] == "2 escales ").astype(np.int64) + (df1["Total d'escales"] == "3 escales ").astype(np.int64)
    return(df1)


Index(['Compagnie', 'Total d'escales', 'Prix', 'Duree', 'Destination',
       'Distance', 'Date_aller', 'Date_retour'],
      dtype='object')

In [213]:
def formatage_prix(liste):
    l=[]
    for i in range (len(liste)):
        c=''
        if len(liste[i]) == 6 :
            c = float(liste[i][0]+liste[i][1]+price[i][2])
        elif len(liste[i]) == 5 : 
            c = float(liste[i][0]+liste[i][1])
        elif len(liste[i]) == 8:
            c = float(liste[i][0]+liste[i][2]+liste[i][3]+ liste[i][4])
        else:
            c = 'NaN'
        l.append(c)
    return l

price = list(df['Prix'])
#df["Prix"] = formatage_prix(price)
df.head()

,Unnamed: 0,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
0,0,Air France,direct,204.0,2h 20min,Oslo,1344.0,2022-04-01,2022-04-08
1,1,Air France,direct,204.0,2h 30min,Oslo,1344.0,2022-04-01,2022-04-08
2,2,Flyr AS,direct,195.0,2h 25min,Oslo,1344.0,2022-04-01,2022-04-08
3,3,Flyr AS,direct,195.0,2h 25min,Oslo,1344.0,2022-04-01,2022-04-08
4,4,Norwegian,direct,186.0,2h 20min,Oslo,1344.0,2022-04-01,2022-04-08


In [254]:
def formatage_duree(liste):
    l=[]
    for i in range (len(liste)):
        l.append(float(duration[i].split('h')[0])+float(duration[i].split('h')[1][1:3])/60)
    return l 

df = pd.read_excel("bdd.xlsx")
df = df.replace('\n','', regex=True)
price = list(df['Prix'])
df["Prix"] = formatage_prix(price)
last = list(df["Duree"])
df["Duree"] = formatage_duree(last)
df.drop(columns = ["Unnamed: 0"])
df = formatage_escale(df)
df[df["Total d'escales"] == 1]

,Unnamed: 0,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
55,55,airBaltic,1,145.0,8.333333,Oslo,1344.00,2022-04-01,2022-04-08
57,57,airBaltic,1,149.0,8.333333,Oslo,1344.00,2022-04-01,2022-04-08
59,59,Eurowings,1,156.0,6.583333,Oslo,1344.00,2022-04-01,2022-04-08
61,61,airBaltic,1,159.0,8.333333,Oslo,1344.00,2022-04-01,2022-04-08
83,83,Plusieurs compagnies,1,141.0,7.083333,Oslo,1344.00,2022-04-01,2022-04-08
...,...,...,...,...,...,...,...,...,...
61339,61339,SWISS,1,1904.0,5.500000,Athenes,2095.52,2022-06-10,2022-06-17
61341,61341,Pegasus Airlines,1,231.0,7.500000,Athenes,2095.52,2022-06-10,2022-06-17
61343,61343,Turkish Airlines,1,294.0,7.333333,Athenes,2095.52,2022-06-10,2022-06-17
61345,61345,SWISS,1,1874.0,5.500000,Athenes,2095.52,2022-06-10,2022-06-17
